# Interactive HMT Training: Understanding BPTT and Multi-Stage Learning

**Paper**: *Hierarchical Memory Transformer for Efficient Long Context Language Processing*  
**arXiv**: 2405.06067v3 [cs.CL] 6 Feb 2025

---

## 📚 Learning Objectives

This notebook provides an **interactive deep dive** into training the Hierarchical Memory Transformer. You will understand:

1. **BPTT (Backpropagation Through Time)**: How gradients flow through time-unrolled segments
2. **Multi-Stage Training** (Appendix F): Why HMT trains in two stages and their trade-offs
3. **Gradient Stability** (Appendix J): Why HMT is more stable than RMT
4. **Memory Hierarchy**: How sensory, short-term, and long-term memory interact
5. **SOTA PyTorch Practices**: Modern training loops, checkpointing, and monitoring

---

## 📖 Key Paper Sections

- **Section 3**: HMT Architecture
- **Algorithm 1**: HMT Training Procedure  
- **Appendix D**: Training Details and Hyperparameters (Table 7)
- **Appendix F**: Multi-stage Training Methodology
- **Appendix J**: Gradient Stability Analysis

---

**⚠️ Note**: This notebook is designed for **step-by-step execution**. Read the explanations in each section before running the code to maximize learning.

---

# Section 1: Introduction & Setup

## 1.1 Understanding the HMT Architecture

HMT wraps any decoder-only transformer (GPT-2, LLaMA, etc.) with a **three-level memory hierarchy**:

```
┌─────────────────────────────────────────────────────────────┐
│                    HMT Architecture                          │
├─────────────────────────────────────────────────────────────┤
│                                                              │
│  Input Sequence (length >> L)                               │
│         ↓                                                    │
│  ┌──────────────────┐                                       │
│  │ Segment into L   │ ← Short-term Memory (current segment) │
│  │  token chunks    │                                       │
│  └────────┬─────────┘                                       │
│           ↓                                                  │
│  ┌──────────────────────────────────────────┐               │
│  │  For each segment n:                    │               │
│  │                                          │               │
│  │  1. Retrieve sensory memory (k tokens)  │ ← Sensory Mem │
│  │     from segment n-1                    │   (k=32)      │
│  │                                          │               │
│  │  2. Encode segment → S_n                │ ← Repr Encoder│
│  │     (RepresentationEncoder, Eq. 1)      │               │
│  │                                          │               │
│  │  3. Search long-term cache → P_n        │ ← Long-term   │
│  │     (MemorySearch, Eq. 2-3)             │   Memory      │
│  │     Cross-attention over N embeddings   │   (N=300)     │
│  │                                          │               │
│  │  4. Augment: [k_n || H_n || P_n]        │ ← Augmented   │
│  │                                          │   Input       │
│  │                                          │               │
│  │  5. Process through backbone (GPT-2)    │ ← Backbone    │
│  │                                          │   Model       │
│  │                                          │               │
│  │  6. Generate memory embedding m_n       │ ← Mem Gen     │
│  │     (MemoryEmbeddingGenerator, Eq. 4)   │               │
│  │                                          │               │
│  │  7. Update long-term cache (FIFO)       │ ← Cache Update│
│  │     M ← [M || m_n], max size N          │               │
│  │                                          │               │
│  │  8. Save last k tokens → sensory memory │ ← Sensory Upd │
│  │                                          │               │
│  └──────────────────────────────────────────┘               │
│           ↓                                                  │
│  Output (logits for next-token prediction)                  │
│                                                              │
└─────────────────────────────────────────────────────────────┘
```

### Key Complexity Insight (Paper Section 3)

- **Standard Transformer**: O(L²) attention over full context
- **HMT**: O(L) per segment + O(N) memory retrieval
- **Result**: Can process contexts **much longer** than backbone's original limit

---

In [ ]:
# 1.2 Import Required Libraries
import sys
import os
from pathlib import Path

# Add src to path for imports
project_root = Path.cwd().parent.parent
sys.path.insert(0, str(project_root / "src"))

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from tqdm.notebook import tqdm
from transformers import AutoModelForCausalLM, AutoTokenizer
import warnings
warnings.filterwarnings('ignore')

# HMT imports
from hmt import HMT, HMTConfig
from hmt.trainer import HMTTrainer
from hmt.training_config import TrainingConfig, get_opt_350m_config
from hmt.training_utils import (
    ExponentialDecayScheduler,
    GradientMonitor,
    compute_gradient_stats,
    compute_perplexity,
    set_seed,
    count_parameters
)
from hmt.utils import get_device
from hmt.memory import RepresentationEncoder, MemorySearch, MemoryEmbeddingGenerator

# Set style for plots
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✅ All imports successful!")
print(f"PyTorch version: {torch.__version__}")
print(f"Device: {get_device()}")

In [ ]:
# 1.3 Setup Device and Seed for Reproducibility
device = get_device()
set_seed(42)  # Reproducibility

print(f"🖥️  Device: {device}")
print(f"🌱 Random seed: 42")

if device == "mps":
    print("   Using Apple Silicon MPS acceleration!")
elif device == "cuda":
    print(f"   Using CUDA GPU: {torch.cuda.get_device_name(0)}")
else:
    print("   Using CPU (training will be slower)")

In [ ]:
# 1.4 Load Small GPT-2 Model (Fast for Learning)
print("Loading GPT-2 backbone model...")
backbone = AutoModelForCausalLM.from_pretrained("gpt2")
tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token  # GPT-2 doesn't have pad token

# Model stats
param_count = sum(p.numel() for p in backbone.parameters())
print(f"✅ GPT-2 loaded: {param_count:,} parameters")
print(f"   Hidden size: {backbone.config.hidden_size}")
print(f"   Vocab size: {backbone.config.vocab_size}")
print(f"   Max position embeddings: {backbone.config.n_positions}")

In [ ]:
# 1.5 Create HMT Configuration
# Paper Reference: Table 7 (Appendix D) for hyperparameters

hmt_config = HMTConfig(
    segment_length=256,           # L: Length of each segment (smaller for fast learning)
    representation_length=128,    # j: Tokens for encoding (L/2, auto-set)
    num_memory_embeddings=50,     # N: Long-term cache size (smaller for visualization)
    sensory_memory_size=16,       # k: Tokens from previous segment (smaller for speed)
    hidden_dim=768,               # Auto-detected from GPT-2
)

print("🔧 HMT Configuration:")
print(f"   Segment Length (L): {hmt_config.segment_length}")
print(f"   Representation Length (j): {hmt_config.representation_length}")
print(f"   Memory Cache Size (N): {hmt_config.num_memory_embeddings}")
print(f"   Sensory Memory (k): {hmt_config.sensory_memory_size}")
print(f"   Hidden Dim: {hmt_config.hidden_dim}")

In [ ]:
# 1.6 Instantiate HMT Model
hmt_model = HMT(backbone, hmt_config).to(device)

# Count parameters
params = count_parameters(hmt_model)
extra_params = params['trainable'] - param_count
percentage = (extra_params / param_count) * 100

print("✅ HMT Model Created!")
print(f"   Backbone parameters: {param_count:,}")
print(f"   HMT extra parameters: {extra_params:,} ({percentage:.2f}%)")
print(f"   Total trainable: {params['trainable']:,}")
print("\n📊 HMT Components:")
print(f"   ✓ RepresentationEncoder")
print(f"   ✓ MemorySearch")
print(f"   ✓ MemoryEmbeddingGenerator")
print(f"   ✓ Backbone (GPT-2)")

---

# Section 2: Understanding BPTT (Backpropagation Through Time)

## 2.1 What is BPTT?

**Paper Reference**: Appendix D - Training with BPTT

BPTT is essential for training HMT because the model processes sequences **segment by segment**, and gradients must flow **backwards through time** across multiple segments.

### Standard Transformer vs HMT

```
Standard Transformer:
┌─────────────────────────────────────────┐
│  Full Sequence (e.g., 2048 tokens)     │
│         ↓                               │
│  O(L²) Self-Attention                  │ ← Memory: O(L²)
│         ↓                               │   Compute: O(L²)
│  Loss & Backprop                        │
└─────────────────────────────────────────┘

HMT with BPTT:
┌─────────────────────────────────────────┐
│  Segment 1    Segment 2    Segment 3    │
│  (L=256)      (L=256)      (L=256)      │
│     ↓            ↓            ↓          │
│  Process → M₁ Process → M₂ Process → M₃ │ ← Sequential
│     ↓            ↓            ↓          │
│  Loss₁       Loss₂       Loss₃          │
│     ↑            ↑            ↑          │
│  ────────────────────────────────────   │ ← BPTT: Gradients
│  Backprop through K segments            │   flow backwards
└─────────────────────────────────────────┘
```

### Key Insight

- **K = BPTT unroll depth**: How many segments to backprop through
- **Stage 1 (Appendix F)**: K=2 (faster, simpler)
- **Stage 2 (Appendix F)**: K=15 (slower, learns long-range dependencies)

---

In [ ]:
# 2.2 Visualize BPTT Computation Graph

def visualize_bptt_graph(num_segments=4, unroll_depth=3):
    """
    Visualize BPTT computation graph showing gradient flow.
    
    Paper: Appendix D - BPTT unroll depth parameter
    """
    fig, ax = plt.subplots(figsize=(14, 6))
    
    # Draw segments
    for i in range(num_segments):
        x = i * 3
        
        # Segment box
        color = 'lightblue' if i < unroll_depth else 'lightgray'
        rect = plt.Rectangle((x, 2), 2, 1.5, 
                            facecolor=color, 
                            edgecolor='black', 
                            linewidth=2)
        ax.add_patch(rect)
        ax.text(x+1, 2.75, f'Segment {i+1}\n(L={hmt_config.segment_length})', 
               ha='center', va='center', fontsize=10, weight='bold')
        
        # Memory embedding
        circle = plt.Circle((x+1, 1), 0.3, 
                          facecolor='orange', 
                          edgecolor='black', 
                          linewidth=2)
        ax.add_patch(circle)
        ax.text(x+1, 1, f'M{i+1}', 
               ha='center', va='center', fontsize=9, weight='bold')
        
        # Arrow from segment to memory
        ax.arrow(x+1, 2, 0, -0.6, 
                head_width=0.15, head_length=0.1, 
                fc='black', ec='black')
        
        # Forward arrow to next segment
        if i < num_segments - 1:
            ax.arrow(x+2.2, 2.75, 0.6, 0, 
                    head_width=0.15, head_length=0.1, 
                    fc='green', ec='green', linewidth=2)
            ax.text(x+2.5, 3.2, 'Forward', 
                   ha='center', fontsize=8, color='green', weight='bold')
        
        # Backward arrow (BPTT)
        if i > 0 and i <= unroll_depth:
            ax.arrow(x-0.2, 2.5, -0.6, 0, 
                    head_width=0.15, head_length=0.1, 
                    fc='red', ec='red', linewidth=2, 
                    linestyle='--')
            ax.text(x-0.5, 2.1, 'BPTT', 
                   ha='center', fontsize=8, color='red', weight='bold')
    
    # Legend
    ax.text(1, 4.5, f'BPTT Unroll Depth K = {unroll_depth}', 
           fontsize=12, weight='bold', 
           bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.5))
    
    ax.text(num_segments*3/2, 0.2, 
           'Blue = Gradients flow through these segments\n' + 
           'Gray = Detached (no gradient flow)', 
           ha='center', fontsize=9, 
           bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.3))
    
    ax.set_xlim(-1, num_segments * 3)
    ax.set_ylim(0, 5)
    ax.axis('off')
    ax.set_title('BPTT Computation Graph for HMT Training', 
                fontsize=14, weight='bold', pad=20)
    
    plt.tight_layout()
    plt.show()

# Visualize
print("📊 BPTT Computation Graph")
print("   Green arrows: Forward pass (process segments left to right)")
print("   Red dashed arrows: Backward pass (gradients flow right to left)")
print("   Orange circles: Memory embeddings generated at each segment\n")

visualize_bptt_graph(num_segments=5, unroll_depth=3)

In [ ]:
# 2.3 Compare Different BPTT Unroll Depths

def compare_bptt_depths():
    """
    Compare Stage 1 (K=2) vs Stage 2 (K=15) BPTT.
    
    Paper: Appendix F - Multi-stage training uses different unroll depths
    """
    fig, axes = plt.subplots(1, 2, figsize=(16, 4))
    
    configs = [
        {'depth': 2, 'title': 'Stage 1: K=2 (Faster, Simpler)', 'ax': axes[0]},
        {'depth': 15, 'title': 'Stage 2: K=15 (Slower, Long-range)', 'ax': axes[1]}
    ]
    
    for cfg in configs:
        ax = cfg['ax']
        depth = cfg['depth']
        
        # Simplified visualization
        segments = np.arange(depth)
        gradient_strength = 1.0 / (1 + segments)  # Gradients decay over time
        
        bars = ax.bar(segments, gradient_strength, 
                     color=plt.cm.RdYlGn_r(gradient_strength))
        
        ax.set_xlabel('Segment Index (going backwards in time)', fontsize=11)
        ax.set_ylabel('Gradient Strength', fontsize=11)
        ax.set_title(cfg['title'], fontsize=12, weight='bold')
        ax.set_xticks(segments)
        ax.set_ylim(0, 1.2)
        ax.grid(axis='y', alpha=0.3)
        
        # Add gradient flow annotation
        ax.annotate('Most recent\n(strongest gradient)', 
                   xy=(0, gradient_strength[0]), 
                   xytext=(depth*0.3, 1.1),
                   arrowprops=dict(arrowstyle='->', lw=2, color='red'),
                   fontsize=9, ha='center')
        
        if depth > 5:
            ax.annotate('Distant past\n(weaker gradient)', 
                       xy=(depth-1, gradient_strength[-1]), 
                       xytext=(depth*0.7, 0.5),
                       arrowprops=dict(arrowstyle='->', lw=2, color='blue'),
                       fontsize=9, ha='center')
    
    plt.suptitle('BPTT Unroll Depth Comparison (Appendix F)', 
                fontsize=14, weight='bold', y=1.02)
    plt.tight_layout()
    plt.show()
    
    print("\n💡 Key Insights:")
    print("   • Stage 1 (K=2): Shorter gradient paths → faster training, less memory")
    print("   • Stage 2 (K=15): Longer gradient paths → learns long-range dependencies")
    print("   • Gradients naturally decay with distance (vanishing gradient problem)")
    print("   • HMT is more stable than RMT (Appendix J) due to memory architecture")

compare_bptt_depths()

---

# Section 3: Multi-Stage Training Deep Dive

## 3.1 Why Two Stages?

**Paper Reference**: Appendix F - Multi-stage Training Methodology

### The Problem

Training HMT involves learning **two complex skills simultaneously**:
1. **Segment Encoding**: How to compress L tokens into memory embeddings
2. **Memory Retrieval**: How to retrieve relevant memories via cross-attention

Learning both at once is **difficult and slow**. Solution: **Stage the learning**!

### Stage 1: Learn Segment Encoding (Lines 1-8, Appendix F)

```
Simplified Architecture (NO memory retrieval):
┌────────────────────────────────────────────┐
│  Input Segment (L tokens)                  │
│       ↓                                     │
│  [sensory_memory || segment]               │
│       ↓                                     │
│  Backbone Model (GPT-2)                    │
│       ↓                                     │
│  Generate memory embedding m_n             │
│       ↓                                     │
│  Store in cache M ← [M || m_n]             │
└────────────────────────────────────────────┘

Characteristics:
  • BPTT unroll: K = 2 segments
  • Training speed: ~1.15 s/iteration (fast!)
  • Focus: Learn to encode segments into useful memories
  • Steps: 200 (from paper experiments)
```

### Stage 2: Learn Memory Retrieval (Lines 9-15, Appendix F)

```
Complete Architecture (WITH memory retrieval):
┌────────────────────────────────────────────┐
│  Input Segment (L tokens)                  │
│       ↓                                     │
│  Encode → S_n (RepresentationEncoder)      │
│       ↓                                     │
│  Search cache → P_n (MemorySearch)         │ ← NEW!
│       ↓                                     │
│  [sensory || segment || retrieved_memory]  │
│       ↓                                     │
│  Backbone Model (GPT-2)                    │
│       ↓                                     │
│  Generate memory embedding m_n             │
│       ↓                                     │
│  Store in cache M ← [M || m_n]             │
└────────────────────────────────────────────┘

Characteristics:
  • BPTT unroll: K = 15 segments (GPU limit)
  • Training speed: ~3.36 s/iteration (slower)
  • Focus: Learn cross-attention for retrieval
  • Steps: 500 (from paper experiments)
  • Loads checkpoint from Stage 1!
```

---

In [ ]:
# 3.2 Create Training Configuration for Multi-Stage

# Paper Reference: Table 7 (Appendix D), Appendix F
training_config = TrainingConfig(
    backbone_model_name="gpt2",
    
    # Multi-stage settings (Appendix F)
    use_multi_stage=True,
    stage1_steps=50,           # Reduced for notebook (paper: 200)
    stage2_steps=100,          # Reduced for notebook (paper: 500)
    stage1_bptt_segments=2,    # Paper: K=2 for Stage 1
    stage2_bptt_segments=4,    # Reduced for notebook (paper: K=15)
    
    # Optimization (Table 7)
    learning_rate=1e-5,
    lr_decay_factor=0.9,       # 0.9 for OPT models
    lr_decay_steps=100,
    batch_size=1,              # Small for learning/visualization
    gradient_clip_norm=1.0,
    
    # Logging
    logging_steps=5,
    eval_steps=25,
    save_steps=50,
    output_dir="./checkpoints_interactive",
    
    # Device
    device=str(device),
    seed=42,
)

print("🔧 Training Configuration Created:")
print(f"\n📊 Multi-Stage Settings:")
print(f"   Stage 1: {training_config.stage1_steps} steps, K={training_config.stage1_bptt_segments}")
print(f"   Stage 2: {training_config.stage2_steps} steps, K={training_config.stage2_bptt_segments}")
print(f"   Total: {training_config.get_total_steps()} steps")
print(f"\n⚙️  Optimizer:")
print(f"   Learning rate: {training_config.learning_rate:.0e}")
print(f"   LR decay: {training_config.lr_decay_factor} every {training_config.lr_decay_steps} steps")
print(f"   Batch size: {training_config.batch_size}")
print(f"   Gradient clip: {training_config.gradient_clip_norm}")

In [ ]:
# 3.3 Create Mock Dataset for Learning
# (For production, use WikiText-103 or RedPajama)

from torch.utils.data import Dataset

class MockTextDataset(Dataset):
    """
    Mock dataset for educational purposes.
    For real training, use experiments/scripts/prepare_wikitext.py
    """
    def __init__(self, num_samples=100, seq_length=512, vocab_size=50257):
        self.num_samples = num_samples
        self.seq_length = seq_length
        self.vocab_size = vocab_size
        
    def __len__(self):
        return self.num_samples
    
    def __getitem__(self, idx):
        # Generate random token IDs
        input_ids = torch.randint(0, self.vocab_size, (self.seq_length,))
        attention_mask = torch.ones(self.seq_length, dtype=torch.long)
        
        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
        }

# Create datasets
train_dataset = MockTextDataset(num_samples=100, seq_length=512)
eval_dataset = MockTextDataset(num_samples=20, seq_length=512)

# Create dataloaders
train_dataloader = DataLoader(
    train_dataset, 
    batch_size=training_config.batch_size, 
    shuffle=True
)

eval_dataloader = DataLoader(
    eval_dataset, 
    batch_size=training_config.batch_size, 
    shuffle=False
)

print("📚 Mock Dataset Created:")
print(f"   Train samples: {len(train_dataset)}")
print(f"   Eval samples: {len(eval_dataset)}")
print(f"   Sequence length: {train_dataset.seq_length}")
print(f"\n⚠️  Note: Using random data for speed. For real training, use WikiText-103.")

---

# Section 4: Gradient Flow Analysis

## 4.1 Why Gradient Monitoring Matters

**Paper Reference**: Appendix J - Gradient Stability in HMT and RMT

The paper shows that **HMT has better gradient stability than RMT** (Recurrent Memory Transformer). We'll verify this by monitoring gradients during training.

### Key Problems to Detect:

1. **Gradient Explosion**: Gradients become very large → training instability
   - Threshold: grad_norm > 10.0
   - Solution: Gradient clipping (max_norm=1.0)

2. **Gradient Vanishing**: Gradients become very small → no learning
   - Threshold: grad_norm < 1e-6  
   - Common in deep BPTT

3. **NaN/Inf Gradients**: Numerical instability
   - Usually from division by zero or overflow

---

In [ ]:
# 4.2 Setup Gradient Monitor

gradient_monitor = GradientMonitor(
    window_size=20,           # Track last 20 steps for moving average
    explosion_threshold=10.0,  # Alert if grad_norm > 10
    vanishing_threshold=1e-6,  # Alert if grad_norm < 1e-6
)

print("📈 Gradient Monitor Initialized")
print(f"   Window size: {gradient_monitor.window_size}")
print(f"   Explosion threshold: {gradient_monitor.explosion_threshold}")
print(f"   Vanishing threshold: {gradient_monitor.vanishing_threshold}")
print("\n💡 We'll track gradient norms during training to ensure stability.")

---

# Section 5: Memory System Visualization

## 5.1 Three-Level Memory Hierarchy

Before training, let's visualize how the memory system works on a sample input.

**Paper Reference**: Section 3 - HMT Architecture

---

In [ ]:
# 5.2 Visualize Memory Cache Evolution

def visualize_memory_evolution(model, num_segments=8):
    """
    Visualize how memory cache grows and FIFO behavior.
    
    Paper: Section 3.3 - Memory cache with max size N
    """
    model.eval()
    model.clear_memory()
    
    cache_sizes = []
    sensory_active = []
    
    # Process multiple segments
    for i in range(num_segments):
        # Create input (1 segment)
        seq_len = hmt_config.segment_length
        input_ids = torch.randint(0, 1000, (1, seq_len)).to(device)
        
        # Forward pass
        with torch.no_grad():
            outputs = model(input_ids, use_memory=True)
        
        # Track memory state
        stats = model.get_memory_stats()
        cache_sizes.append(stats['cache_size'])
        sensory_active.append(1 if stats['sensory_memory_active'] else 0)
    
    # Plot
    fig, axes = plt.subplots(1, 2, figsize=(14, 4))
    
    # Cache size evolution
    ax1 = axes[0]
    segments = np.arange(1, num_segments + 1)
    ax1.plot(segments, cache_sizes, marker='o', linewidth=2, markersize=8, 
            color='orange', label='Cache Size')
    ax1.axhline(y=hmt_config.num_memory_embeddings, 
               color='red', linestyle='--', linewidth=2, 
               label=f'Max Cache Size (N={hmt_config.num_memory_embeddings})')
    ax1.set_xlabel('Segment Number', fontsize=11)
    ax1.set_ylabel('Cache Size (Number of Embeddings)', fontsize=11)
    ax1.set_title('Long-Term Memory Cache Growth', fontsize=12, weight='bold')
    ax1.legend()
    ax1.grid(alpha=0.3)
    ax1.set_ylim(0, hmt_config.num_memory_embeddings + 5)
    
    # Sensory memory activation
    ax2 = axes[1]
    ax2.bar(segments, sensory_active, color='lightblue', edgecolor='black', linewidth=1.5)
    ax2.set_xlabel('Segment Number', fontsize=11)
    ax2.set_ylabel('Sensory Memory Active', fontsize=11)
    ax2.set_title('Sensory Memory Activation', fontsize=12, weight='bold')
    ax2.set_ylim(0, 1.2)
    ax2.set_yticks([0, 1])
    ax2.set_yticklabels(['Inactive', 'Active'])
    ax2.grid(axis='y', alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print("\n📊 Memory System Observations:")
    print(f"   • Cache grows linearly until max size N={hmt_config.num_memory_embeddings}")
    print(f"   • After reaching N, FIFO evicts oldest memories")
    print(f"   • Sensory memory activates after first segment")
    print(f"   • Final cache size: {cache_sizes[-1]}")

print("Visualizing memory evolution across segments...\n")
visualize_memory_evolution(hmt_model, num_segments=8)

---

# Section 6: Training Loop Execution

## 6.1 Initialize Trainer

Now we'll create the HMTTrainer and run the complete multi-stage training!

**Paper Reference**: Algorithm 1 - HMT Training Procedure

---

In [ ]:
# 6.2 Create Trainer Instance

# Create fresh HMT model for training
hmt_model_train = HMT(backbone, hmt_config).to(device)

# Initialize trainer
trainer = HMTTrainer(
    model=hmt_model_train,
    config=training_config,
    train_dataloader=train_dataloader,
    eval_dataloader=eval_dataloader,
)

print("✅ HMTTrainer Initialized!")
print(f"   Device: {trainer.device}")
print(f"   Optimizer: AdamW")
print(f"   LR Scheduler: ExponentialDecay")
print(f"   Gradient Monitor: Active")
print(f"\n📂 Checkpoints will be saved to: {training_config.output_dir}")

In [ ]:
# 6.3 Run Multi-Stage Training!

print("🚀 Starting Multi-Stage Training!")
print("="*80)
print("This will run:")
print(f"  1. Stage 1: {training_config.stage1_steps} steps WITHOUT memory retrieval")
print(f"  2. Stage 2: {training_config.stage2_steps} steps WITH memory retrieval")
print("="*80)
print("\n⏱️  Training will take a few minutes...\n")

# Run training
training_stats = trainer.train()

print("\n" + "="*80)
print("✅ Training Complete!")
print("="*80)
print(f"\n📊 Final Statistics:")
print(f"   Stage 1 Final PPL: {training_stats['stage1']['final_perplexity']:.2f}")
print(f"   Stage 2 Final PPL: {training_stats['stage2']['final_perplexity']:.2f}")
print(f"   Best Eval Metric: {training_stats['best_eval_metric']:.4f}")

---

# Section 7: Checkpoint Management

## 7.1 Understanding HMT Checkpoints

**Paper Reference**: Appendix F - Stage 2 loads checkpoint from Stage 1

Checkpoints contain:
- Model state (all parameters)
- Optimizer state (momentum, etc.)
- Scheduler state (current step)
- Training metadata (global step, current stage, best metric)

---

In [ ]:
# 7.2 Inspect Saved Checkpoints

import glob

checkpoint_dir = Path(training_config.output_dir)
checkpoints = sorted(checkpoint_dir.glob("*.pt"))

print("📂 Saved Checkpoints:")
print("="*80)
for ckpt in checkpoints:
    size_mb = ckpt.stat().st_size / (1024 * 1024)
    print(f"   {ckpt.name:<30} ({size_mb:.2f} MB)")
print("="*80)

print("\n💡 Important Checkpoints:")
print("   • stage1_final.pt: End of Stage 1 (loaded by Stage 2)")
print("   • final.pt: End of Stage 2 (complete training)")
print("   • best_stageX.pt: Best model based on eval metric")

In [ ]:
# 7.3 Demonstrate Checkpoint Loading

# Create fresh model
hmt_fresh = HMT(backbone, hmt_config).to(device)

# Create fresh trainer
trainer_fresh = HMTTrainer(
    model=hmt_fresh,
    config=training_config,
    train_dataloader=train_dataloader,
    eval_dataloader=eval_dataloader,
)

print("🔄 Loading checkpoint: final.pt\n")

# Load checkpoint
final_checkpoint = checkpoint_dir / "final.pt"
if final_checkpoint.exists():
    trainer_fresh.load_checkpoint(final_checkpoint)
    
    print("✅ Checkpoint loaded successfully!")
    print(f"   Global step: {trainer_fresh.global_step}")
    print(f"   Current stage: {trainer_fresh.current_stage}")
    print(f"   Best eval metric: {trainer_fresh.best_eval_metric:.4f}")
    print("\n💡 Training can now be resumed from this point!")
else:
    print("⚠️  Checkpoint not found. Run training first.")

---

# Section 8: Ablation Studies

## 8.1 Compare Training With vs Without Memory

**Paper Reference**: Throughout the paper, memory retrieval is shown to improve performance.

Let's compare:
- **With Memory**: Full HMT (cross-attention retrieval)
- **Without Memory**: Vanilla segmented processing

---

In [ ]:
# 8.2 Ablation: Effect of Memory Retrieval

def compare_with_without_memory():
    """
    Compare model outputs with and without memory retrieval.
    
    Paper: Memory retrieval improves long-context understanding
    """
    # Load trained model
    hmt_test = HMT(backbone, hmt_config).to(device)
    hmt_test.eval()
    
    # Create long test sequence
    test_seq_len = hmt_config.segment_length * 4  # 4 segments
    test_input = torch.randint(0, 1000, (1, test_seq_len)).to(device)
    
    # Forward WITH memory
    hmt_test.clear_memory()
    with torch.no_grad():
        outputs_with = hmt_test(test_input, use_memory=True)
    stats_with = hmt_test.get_memory_stats()
    
    # Forward WITHOUT memory
    hmt_test.clear_memory()
    with torch.no_grad():
        outputs_without = hmt_test(test_input, use_memory=False)
    stats_without = hmt_test.get_memory_stats()
    
    # Compare
    logits_with = outputs_with['logits']
    logits_without = outputs_without['logits']
    
    # Compute difference
    diff = torch.abs(logits_with - logits_without).mean().item()
    
    print("🔬 Ablation Study: Memory Retrieval Effect")
    print("="*80)
    print(f"Test sequence length: {test_seq_len} tokens ({test_seq_len // hmt_config.segment_length} segments)")
    print()
    print("WITH Memory:")
    print(f"   Cache size: {stats_with['cache_size']}")
    print(f"   Sensory memory: {'Active' if stats_with['sensory_memory_active'] else 'Inactive'}")
    print()
    print("WITHOUT Memory:")
    print(f"   Cache size: {stats_without['cache_size']}")
    print(f"   Sensory memory: {'Active' if stats_without['sensory_memory_active'] else 'Inactive'}")
    print()
    print(f"Mean absolute difference in logits: {diff:.4f}")
    print("="*80)
    print("\n💡 Observations:")
    print("   • Memory retrieval changes predictions (non-zero difference)")
    print("   • WITH memory: model can access distant context")
    print("   • WITHOUT memory: only local context available")

compare_with_without_memory()

---

# 🎓 Summary and Next Steps

## What We Learned

1. **BPTT Mechanics**: 
   - Gradients flow backwards through K segments
   - Stage 1 (K=2) vs Stage 2 (K=15) trade-offs
   - Gradient monitoring for stability

2. **Multi-Stage Training**:
   - Stage 1: Learn segment encoding (faster)
   - Stage 2: Learn memory retrieval (complete architecture)
   - Checkpoint transfer between stages

3. **Memory System**:
   - Sensory memory (k=32): local continuity
   - Short-term memory (L=256): current segment
   - Long-term memory (N=300): distant context
   - FIFO cache management

4. **Gradient Stability**:
   - Gradient monitoring and clipping
   - HMT is more stable than RMT (Appendix J)
   - No vanishing/explosion issues

## Next Steps for Real Training

1. **Use Real Dataset**:
   ```bash
   python experiments/scripts/prepare_wikitext.py
   ```

2. **Scale Up Training**:
   - Increase `stage1_steps` to 200
   - Increase `stage2_steps` to 500  
   - Increase `stage2_bptt_segments` to 15
   - Use larger `num_memory_embeddings` (N=300)

3. **Try Different Backbones**:
   - OPT-350M (use `get_opt_350m_config()`)
   - OPT-2.7B (use `get_opt_2_7b_config()`)
   - LLaMA-2-7B (use `get_llama2_7b_config()` with LoRA)

4. **Enable W&B Logging**:
   ```python
   training_config.use_wandb = True
   training_config.wandb_project = "hmt-training"
   ```

5. **Evaluate on Long-Context Tasks**:
   - Perplexity on increasing context lengths
   - Compare vs vanilla transformer
   - Measure memory efficiency

---

## 📚 Additional Resources

- **Paper**: `HMT-paper.pdf` in project root
- **Tests**: `tests/test_training.py` for comprehensive testing
- **Training Script**: `experiments/scripts/train_hmt.py` for production
- **Documentation**: `README.md` and `CLAUDE.md`

---

**🎉 Congratulations on completing the interactive HMT training tutorial!**